In [14]:
# Les tests pour les modeles de création d'objets Java (Hanoi, Toulouse, Nantes) testCreationObjetsJavaHanoi.gaml
import os
import csv

gtfs_dir = r"C:\Users\tiend\Downloads\Toulouse_GTFS"  # Chemin vers le répertoire GTFS

def detect_separator(file_path):
    """Détecte le séparateur CSV (comme dans le code Java)."""
    if not os.path.exists(file_path):
        return ','
    
    with open(file_path, 'r', encoding='utf-8') as f:
        first_line = f.readline().strip()
        if not first_line:
            return ','
        
        # Compte virgules/points-virgules hors guillemets (logique Java)
        comma_count = 0
        semicolon_count = 0
        in_quotes = False
        
        for c in first_line:
            if c == '"':
                in_quotes = not in_quotes
            if not in_quotes:
                if c == ',':
                    comma_count += 1
                elif c == ';':
                    semicolon_count += 1
        
        return ';' if semicolon_count > comma_count else ','

def count_lines_without_header(file_path):
    """Compte les lignes hors header dans un fichier texte."""
    if not os.path.exists(file_path):
        return "Absent"
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            if len(lines) <= 1:  # Seulement header ou fichier vide
                return 0
            return len([line for line in lines[1:] if line.strip()])  # Ignore les lignes vides
    except:
        return "Erreur"

def get_unique_ids(file_path, id_column):
    """Retourne l'ensemble des IDs uniques dans une colonne."""
    if not os.path.exists(file_path):
        return set()
    
    try:
        separator = detect_separator(file_path)
        with open(file_path, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f, delimiter=separator)
            if id_column not in reader.fieldnames:
                return set()
            return set(row[id_column].strip().replace('"', '').replace("'", '') 
                      for row in reader if row[id_column].strip())
    except:
        return set()

def count_shapes_created_java_style():
    """Simule la logique Java pour compter les shapes créés."""
    shapes_txt = os.path.join(gtfs_dir, "shapes.txt")
    
    if not os.path.exists(shapes_txt):
        return "Absent"
    
    try:
        separator = detect_separator(shapes_txt)
        valid_shapes = set()
        total_lines = 0
        error_lines = 0
        
        with open(shapes_txt, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f, delimiter=separator)
            
            # Vérification des colonnes requises
            required_cols = ['shape_id', 'shape_pt_lat', 'shape_pt_lon']
            if not all(col in reader.fieldnames for col in required_cols):
                return "Colonnes manquantes"
            
            for row in reader:
                total_lines += 1
                try:
                    # Simulation exacte de la logique Java
                    shape_id = int(row['shape_id'].strip())  # Integer.parseInt()
                    lat = float(row['shape_pt_lat'].strip())  # Double.parseDouble()
                    lon = float(row['shape_pt_lon'].strip())  # Double.parseDouble()
                    
                    # Si on arrive ici, la ligne est valide (comme en Java)
                    valid_shapes.add(shape_id)
                    
                except (ValueError, KeyError):
                    # Équivalent du catch Exception en Java
                    error_lines += 1
                    continue
        
        return len(valid_shapes)
    
    except:
        return "Erreur"

def analyze_gtfs():
    """Analyse complète GTFS avec les mêmes indicateurs que le code Java."""
    
    print("---- Récapitulatif création objets GTFS ----")
    print(f"Répertoire GTFS analysé : {gtfs_dir}")
    
    # 1. Nombre de stops lus dans stops.txt
    stops_txt = os.path.join(gtfs_dir, "stops.txt")
    stops_lus = count_lines_without_header(stops_txt)
    print(f"Nombre de stops lus dans stops.txt          : {stops_lus}   # Tous les arrêts déclarés dans stops.txt")
    
    # 2. Nombre de stops créés (présents dans stops.txt ET utilisés dans stop_times.txt)
    if stops_lus != "Absent":
        stops_declared = get_unique_ids(stops_txt, "stop_id")
        stops_used = get_unique_ids(os.path.join(gtfs_dir, "stop_times.txt"), "stop_id")
        stops_crees = len(stops_declared & stops_used) if stops_declared and stops_used else 0
    else:
        stops_crees = "Absent"
    print(f"Nombre de stops créés (stopsMap)            : {stops_crees}   # Arrêts utilisés (présents aussi dans stop_times.txt)")
    
    # 3. Nombre de trips créés
    trips_txt = os.path.join(gtfs_dir, "trips.txt")
    if os.path.exists(trips_txt):
        trips_ids = get_unique_ids(trips_txt, "trip_id")
        trips_crees = len(trips_ids)
    else:
        trips_crees = "Absent"
    print(f"Nombre de trips créés (tripsMap)            : {trips_crees}   # trip_id uniques dans trips.txt")
    
    # 4. Nombre de shapes lus dans shapes.txt
    shapes_txt = os.path.join(gtfs_dir, "shapes.txt")
    shapes_lus = count_lines_without_header(shapes_txt)
    print(f"Nombre de shapes lus dans shapes.txt        : {shapes_lus}   # Nombre de lignes - header dans shapes.txt ('Absent' si non présent)")
    
    # 5. Nombre de shapes créés (avec logique Java)
    shapes_crees = count_shapes_created_java_style()
    print(f"Nombre de shapes créés (shapesMap)          : {shapes_crees}   # Nombre de shape_id uniques (shapes uniques, 'Absent' si shapes.txt absent)")
    
    print("--------------------------------------------")



if __name__ == "__main__":
    analyze_gtfs()

---- Récapitulatif création objets GTFS ----
Répertoire GTFS analysé : C:\Users\tiend\Downloads\Toulouse_GTFS
Nombre de stops lus dans stops.txt          : 5569   # Tous les arrêts déclarés dans stops.txt
Nombre de stops créés (stopsMap)            : 3759   # Arrêts utilisés (présents aussi dans stop_times.txt)
Nombre de trips créés (tripsMap)            : 38719   # trip_id uniques dans trips.txt
Nombre de shapes lus dans shapes.txt        : 270853   # Nombre de lignes - header dans shapes.txt ('Absent' si non présent)
Nombre de shapes créés (shapesMap)          : 453   # Nombre de shape_id uniques (shapes uniques, 'Absent' si shapes.txt absent)
--------------------------------------------


In [15]:
# Les tests pour les modeles de création d'objets Java (Hanoi, Toulouse, Nantes) testCreationObjetsJavaHanoi.gaml
import os

import os
import csv

gtfs_dir = r"C:\Users\tiend\Downloads\GTFS_Hanoi"  # Chemin vers le répertoire GTFS

def detect_separator(file_path):
    """Détecte le séparateur CSV (comme dans le code Java)."""
    if not os.path.exists(file_path):
        return ','
    
    with open(file_path, 'r', encoding='utf-8') as f:
        first_line = f.readline().strip()
        if not first_line:
            return ','
        
        # Compte virgules/points-virgules hors guillemets (logique Java)
        comma_count = 0
        semicolon_count = 0
        in_quotes = False
        
        for c in first_line:
            if c == '"':
                in_quotes = not in_quotes
            if not in_quotes:
                if c == ',':
                    comma_count += 1
                elif c == ';':
                    semicolon_count += 1
        
        return ';' if semicolon_count > comma_count else ','

def count_lines_without_header(file_path):
    """Compte les lignes hors header dans un fichier texte."""
    if not os.path.exists(file_path):
        return "Absent"
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            if len(lines) <= 1:  # Seulement header ou fichier vide
                return 0
            return len([line for line in lines[1:] if line.strip()])  # Ignore les lignes vides
    except:
        return "Erreur"

def get_unique_ids(file_path, id_column):
    """Retourne l'ensemble des IDs uniques dans une colonne."""
    if not os.path.exists(file_path):
        return set()
    
    try:
        separator = detect_separator(file_path)
        with open(file_path, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f, delimiter=separator)
            if id_column not in reader.fieldnames:
                return set()
            return set(row[id_column].strip().replace('"', '').replace("'", '') 
                      for row in reader if row[id_column].strip())
    except:
        return set()

def count_shapes_created_java_style():
    """Simule la logique Java pour compter les shapes créés."""
    shapes_txt = os.path.join(gtfs_dir, "shapes.txt")
    
    if not os.path.exists(shapes_txt):
        return "Absent"
    
    try:
        separator = detect_separator(shapes_txt)
        valid_shapes = set()
        total_lines = 0
        error_lines = 0
        
        with open(shapes_txt, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f, delimiter=separator)
            
            # Vérification des colonnes requises
            required_cols = ['shape_id', 'shape_pt_lat', 'shape_pt_lon']
            if not all(col in reader.fieldnames for col in required_cols):
                return "Colonnes manquantes"
            
            for row in reader:
                total_lines += 1
                try:
                    # Simulation exacte de la logique Java
                    shape_id = int(row['shape_id'].strip())  # Integer.parseInt()
                    lat = float(row['shape_pt_lat'].strip())  # Double.parseDouble()
                    lon = float(row['shape_pt_lon'].strip())  # Double.parseDouble()
                    
                    # Si on arrive ici, la ligne est valide (comme en Java)
                    valid_shapes.add(shape_id)
                    
                except (ValueError, KeyError):
                    # Équivalent du catch Exception en Java
                    error_lines += 1
                    continue
        
        return len(valid_shapes)
    
    except:
        return "Erreur"

def analyze_gtfs():
    """Analyse complète GTFS avec les mêmes indicateurs que le code Java."""
    
    print("---- Récapitulatif création objets GTFS ----")
    print(f"Répertoire GTFS analysé : {gtfs_dir}")
    
    # 1. Nombre de stops lus dans stops.txt
    stops_txt = os.path.join(gtfs_dir, "stops.txt")
    stops_lus = count_lines_without_header(stops_txt)
    print(f"Nombre de stops lus dans stops.txt          : {stops_lus}   # Tous les arrêts déclarés dans stops.txt")
    
    # 2. Nombre de stops créés (présents dans stops.txt ET utilisés dans stop_times.txt)
    if stops_lus != "Absent":
        stops_declared = get_unique_ids(stops_txt, "stop_id")
        stops_used = get_unique_ids(os.path.join(gtfs_dir, "stop_times.txt"), "stop_id")
        stops_crees = len(stops_declared & stops_used) if stops_declared and stops_used else 0
    else:
        stops_crees = "Absent"
    print(f"Nombre de stops créés (stopsMap)            : {stops_crees}   # Arrêts utilisés (présents aussi dans stop_times.txt)")
    
    # 3. Nombre de trips créés
    trips_txt = os.path.join(gtfs_dir, "trips.txt")
    if os.path.exists(trips_txt):
        trips_ids = get_unique_ids(trips_txt, "trip_id")
        trips_crees = len(trips_ids)
    else:
        trips_crees = "Absent"
    print(f"Nombre de trips créés (tripsMap)            : {trips_crees}   # trip_id uniques dans trips.txt")
    
    # 4. Nombre de shapes lus dans shapes.txt
    shapes_txt = os.path.join(gtfs_dir, "shapes.txt")
    shapes_lus = count_lines_without_header(shapes_txt)
    print(f"Nombre de shapes lus dans shapes.txt        : {shapes_lus}   # Nombre de lignes - header dans shapes.txt ('Absent' si non présent)")
    
    # 5. Nombre de shapes créés (avec logique Java)
    shapes_crees = count_shapes_created_java_style()
    print(f"Nombre de shapes créés (shapesMap)          : {shapes_crees}   # Nombre de shape_id uniques (shapes uniques, 'Absent' si shapes.txt absent)")
    
    print("--------------------------------------------")



if __name__ == "__main__":
    analyze_gtfs()

---- Récapitulatif création objets GTFS ----
Répertoire GTFS analysé : C:\Users\tiend\Downloads\GTFS_Hanoi
Nombre de stops lus dans stops.txt          : 7670   # Tous les arrêts déclarés dans stops.txt
Nombre de stops créés (stopsMap)            : 7670   # Arrêts utilisés (présents aussi dans stop_times.txt)
Nombre de trips créés (tripsMap)            : 6713   # trip_id uniques dans trips.txt
Nombre de shapes lus dans shapes.txt        : Absent   # Nombre de lignes - header dans shapes.txt ('Absent' si non présent)
Nombre de shapes créés (shapesMap)          : Absent   # Nombre de shape_id uniques (shapes uniques, 'Absent' si shapes.txt absent)
--------------------------------------------


In [16]:
import os
import csv

gtfs_dir = r"C:\Users\tiend\Downloads\gtfs-tan"  # Chemin vers le répertoire GTFS

def detect_separator(file_path):
    """Détecte le séparateur CSV (comme dans le code Java)."""
    if not os.path.exists(file_path):
        return ','
    
    with open(file_path, 'r', encoding='utf-8') as f:
        first_line = f.readline().strip()
        if not first_line:
            return ','
        
        # Compte virgules/points-virgules hors guillemets (logique Java)
        comma_count = 0
        semicolon_count = 0
        in_quotes = False
        
        for c in first_line:
            if c == '"':
                in_quotes = not in_quotes
            if not in_quotes:
                if c == ',':
                    comma_count += 1
                elif c == ';':
                    semicolon_count += 1
        
        return ';' if semicolon_count > comma_count else ','

def count_lines_without_header(file_path):
    """Compte les lignes hors header dans un fichier texte."""
    if not os.path.exists(file_path):
        return "Absent"
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            if len(lines) <= 1:  # Seulement header ou fichier vide
                return 0
            return len([line for line in lines[1:] if line.strip()])  # Ignore les lignes vides
    except:
        return "Erreur"

def get_unique_ids(file_path, id_column):
    """Retourne l'ensemble des IDs uniques dans une colonne."""
    if not os.path.exists(file_path):
        return set()
    
    try:
        separator = detect_separator(file_path)
        with open(file_path, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f, delimiter=separator)
            if id_column not in reader.fieldnames:
                return set()
            return set(row[id_column].strip().replace('"', '').replace("'", '') 
                      for row in reader if row[id_column].strip())
    except:
        return set()

def count_shapes_created_java_style():
    """Simule la logique Java pour compter les shapes créés."""
    shapes_txt = os.path.join(gtfs_dir, "shapes.txt")
    
    if not os.path.exists(shapes_txt):
        return "Absent"
    
    try:
        separator = detect_separator(shapes_txt)
        valid_shapes = set()
        total_lines = 0
        error_lines = 0
        
        with open(shapes_txt, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f, delimiter=separator)
            
            # Vérification des colonnes requises
            required_cols = ['shape_id', 'shape_pt_lat', 'shape_pt_lon']
            if not all(col in reader.fieldnames for col in required_cols):
                return "Colonnes manquantes"
            
            for row in reader:
                total_lines += 1
                try:
                    # Simulation exacte de la logique Java
                    shape_id = int(row['shape_id'].strip())  # Integer.parseInt()
                    lat = float(row['shape_pt_lat'].strip())  # Double.parseDouble()
                    lon = float(row['shape_pt_lon'].strip())  # Double.parseDouble()
                    
                    # Si on arrive ici, la ligne est valide (comme en Java)
                    valid_shapes.add(shape_id)
                    
                except (ValueError, KeyError):
                    # Équivalent du catch Exception en Java
                    error_lines += 1
                    continue
        
        return len(valid_shapes)
    
    except:
        return "Erreur"

def analyze_gtfs():
    """Analyse complète GTFS avec les mêmes indicateurs que le code Java."""
    
    print("---- Récapitulatif création objets GTFS ----")
    print(f"Répertoire GTFS analysé : {gtfs_dir}")
    
    # 1. Nombre de stops lus dans stops.txt
    stops_txt = os.path.join(gtfs_dir, "stops.txt")
    stops_lus = count_lines_without_header(stops_txt)
    print(f"Nombre de stops lus dans stops.txt          : {stops_lus}   # Tous les arrêts déclarés dans stops.txt")
    
    # 2. Nombre de stops créés (présents dans stops.txt ET utilisés dans stop_times.txt)
    if stops_lus != "Absent":
        stops_declared = get_unique_ids(stops_txt, "stop_id")
        stops_used = get_unique_ids(os.path.join(gtfs_dir, "stop_times.txt"), "stop_id")
        stops_crees = len(stops_declared & stops_used) if stops_declared and stops_used else 0
    else:
        stops_crees = "Absent"
    print(f"Nombre de stops créés (stopsMap)            : {stops_crees}   # Arrêts utilisés (présents aussi dans stop_times.txt)")
    
    # 3. Nombre de trips créés
    trips_txt = os.path.join(gtfs_dir, "trips.txt")
    if os.path.exists(trips_txt):
        trips_ids = get_unique_ids(trips_txt, "trip_id")
        trips_crees = len(trips_ids)
    else:
        trips_crees = "Absent"
    print(f"Nombre de trips créés (tripsMap)            : {trips_crees}   # trip_id uniques dans trips.txt")
    
    # 4. Nombre de shapes lus dans shapes.txt
    shapes_txt = os.path.join(gtfs_dir, "shapes.txt")
    shapes_lus = count_lines_without_header(shapes_txt)
    print(f"Nombre de shapes lus dans shapes.txt        : {shapes_lus}   # Nombre de lignes - header dans shapes.txt ('Absent' si non présent)")
    
    # 5. Nombre de shapes créés (avec logique Java)
    shapes_crees = count_shapes_created_java_style()
    print(f"Nombre de shapes créés (shapesMap)          : {shapes_crees}   # Nombre de shape_id uniques (shapes uniques, 'Absent' si shapes.txt absent)")
    
    print("--------------------------------------------")



if __name__ == "__main__":
    analyze_gtfs()

---- Récapitulatif création objets GTFS ----
Répertoire GTFS analysé : C:\Users\tiend\Downloads\gtfs-tan
Nombre de stops lus dans stops.txt          : 3687   # Tous les arrêts déclarés dans stops.txt
Nombre de stops créés (stopsMap)            : 2556   # Arrêts utilisés (présents aussi dans stop_times.txt)
Nombre de trips créés (tripsMap)            : 30548   # trip_id uniques dans trips.txt
Nombre de shapes lus dans shapes.txt        : 14608   # Nombre de lignes - header dans shapes.txt ('Absent' si non présent)
Nombre de shapes créés (shapesMap)          : 261   # Nombre de shape_id uniques (shapes uniques, 'Absent' si shapes.txt absent)
--------------------------------------------
